In [9]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

 ········


In [23]:
def solve(ssh, dataset="BR8.txt", inst_id=0, time=10, a=4.0, b=1.0, g=0.2, p=0.04):
    command = "/home/iaraya/clp/BSG_CLP /home/iaraya/clp/problems/clp/benchs/BR/" + \
                dataset +" -i "+ str(inst_id) +" -t "+str(time)+ \
                " --alpha="+str(a)+" --beta="+str(b)+" --gamma="+str(g)+" -p "+str(p)
    print(command)
    stdin, stdout, stderr = ssh.exec_command(command)
    lines = stdout.readlines()
    return float(lines[-1])

In [31]:
solve(ssh)

/home/iaraya/clp/BSG_CLP /home/iaraya/clp/problems/clp/benchs/BR/BR8.txt -i 0 -t 10 --alpha=4.0 --beta=1.0 --gamma=0.2 -p 0.04


'94.667217\n'

In [28]:
import fwk4exps.speculative_monitor as fwk
f4e = fwk.SpeculativeMonitor(cpu_count=3)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [29]:
def parameter_tuning(S, param, param_values):
    original_value = S.params[param]
    original_name = S.name
    params = S.params.copy()
    for value in param_values:
        params[param]=value
        if original_value == value: 
            continue
        else: 
            S2 = fwk.Strategy(original_name, S.pathExe, S.args, params)
        S = f4e.bestStrategy(S, S2)
    return S

In [ ]:
def experimentalDesign():

    S = fwk.Strategy('BSG_CLP', './BSG_CLP', '--alpha={a} --beta={b} --gamma={g} -p {p}', {"a": 0.0, "b": 0.0, "g": 0.0, "p": 0.0})

    S = parameter_tuning(S, "a", [0.0, 1.0, 2.0, 4.0, 8.0])
    S = parameter_tuning(S, "b", [0.0, 0.5, 1.0, 2.0, 4.0])
    S = parameter_tuning(S, "g", [0.0, 0.1, 0.2, 0.3, 0.4])
    S = parameter_tuning(S, "p", [0.0, 0.1, 0.2, 0.3, 0.4])

    f4e.output = S.name + " " + str(tuple(S.params.values())) + " "
    f4e.terminate()    

f4e.speculative_execution(experimentalDesign, 'instancesCLP-shuf.txt')